# Header

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor
from sklearn.linear_model import LinearRegression
import os

exec(open("../header.py").read())

Header initialized


# Import

In [52]:
raw_train_data = pd.read_csv(processed_root("03-bag-of-words/bow_train_data.csv"))
raw_val_data = pd.read_csv(processed_root("03-bag-of-words/bow_val_data.csv"))
raw_test_data = pd.read_csv(processed_root("03-bag-of-words/bow_test_data.csv"))

In [53]:
X_train = raw_train_data\
    .drop(['poetry_text', 'poetry_author'], axis = 1)
X_val = raw_val_data\
    .drop(['poetry_text', 'poetry_author'], axis = 1)
y_train = raw_train_data['poetry_author']
y_val = raw_val_data['poetry_author']

# Model

In [54]:
dt = DecisionTreeClassifier(max_depth = 5, class_weight = "balanced")

# Train

In [55]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

# Evaluate

In [56]:
# Training
y_pred_train = dt.predict(X_train)
train_accuracy = np.mean(y_pred_train == y_train) * 100
print("Training Accuracy: %.2f%%"%(train_accuracy))

# Validation
y_pred_val = dt.predict(X_val)
val_accuracy = np.mean(y_pred_val == y_val) * 100
print("Validation Accuracy: %.2f%%"%(val_accuracy))

Training Accuracy: 39.61%
Validation Accuracy: 30.47%
